In [ ]:
import os
import pathlib
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import models
import wave
from scipy.io.wavfile import read
from sklearn.model_selection import train_test_split
import librosa
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import Sequential
import sounddevice as sd
from scipy.io.wavfile import write
import matplotlib.pyplot as plt
from keras.models import Model
from keras.layers import  Conv1D,  GRU, Dropout, Input, Dense, Conv2D, Reshape, MaxPooling2D
import random
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from scipy import linalg
sample_rate = 16000
seconds = 3
case=0
model = tf.keras.Model()
model=tf.keras.models.load_model('../keyword_model_1D_CRNN')


In [ ]:
recording = sd.rec(int(seconds * fs), samplerate=sample_rate, channels=1)
sd.wait()
first_audio = np.squeeze(recording)

long_audio=first_audio
while True:
    if case == 150:
        break
    case+= 1
    print("Say Now: ")
    recording = sd.rec(int(1 * fs), samplerate=fs, channels=1)
    sd.wait()
    
    audio = np.squeeze(recording)
    long_audio = np.delete(long_audio , [index for index in range(int(np.shape(audio)[0]))] )
    long_audio = np.hstack((long_audio, audio))
    mel_spec = librosa.feature.melspectrogram(y=long_audio, sr=sample_rate, n_fft=512, hop_length=160, n_mels=48, fmax=8000)
    mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max).T
    mel_spec_db_norm = (mel_spec_db - np.min(mel_spec_db)) / (np.max(mel_spec_db) - np.min(mel_spec_db))

    prediction = model.predict(np.expand_dims(mel_spec_db_norm, axis=0), batch_size=32)
    print(prediction)
    if prediction[:, 1] > 0.50:
        print(f"Wake Word Detected for ({case})")
        print("Confidence:", prediction[:, 1])
        
    
    else:
        print(f"Wake Word NOT Detected")
        print("Confidence:", prediction[:, 0])
        